In [ ]:
'''
1) Прочитайте главы 2-3 из книги "Spark: The Definitive Guide"
2) Установите библиотеки numpy, pandas, matplotlib и seasborn.
Для этого введите в консоли:

source ~/Work/bin/activate
pip3 install numpy
pip3 install pandas
pip3 install matplotlib
pip3 install seaborn

3) Скачайте датасет TikTok_songs_2022.csv по ссылке https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022
4) Выполните задания 1-5 с помощью SQL и методов DataFrame.
5) Ответьте на вопросы в задании 6.
'''

In [ ]:
# Создаем SparkSession
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkApp') \
                    .getOrCreate()

In [ ]:
# Загружаем датасет https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022
tiktokData2022 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("path to csv file")

tiktokData2022.createOrReplaceTempView("TikTok_songs_2022")

In [ ]:
# 0: Получите таблицу из  5 названий треков с их исполнителями.

# +--------------------+-------------+
# |          track_name|  artist_name|
# +--------------------+-------------+
# |Running Up That H...|    Kate Bush|
# |           As It Was| Harry Styles|
# |             Sunroof|  Nicky Youre|
# |          Heat Waves|Glass Animals|
# |     About Damn Time|        Lizzo|
# +--------------------+-------------+

from pyspark.sql.functions import desc

sqlWay = spark.sql("""
SELECT track_name,artist_name
FROM TikTok_songs_2022
LIMIT 5
""")

dataFrameWay = tiktokData2022\
    .select(
    "track_name",
    "artist_name")\
    .limit(5)

sqlWay.show()
dataFrameWay.show()

In [ ]:
# 1: Получите список исполнителей с популярностью больше 90%

# ['Drake', 'Harry Styles', 'The Weeknd']

sqlWay = spark.sql("""

""")

dataFrameWay = ...

print(sqlWay.rdd.map(lambda x: x[0]).collect())
print(dataFrameWay.rdd.map(lambda x: x[0]).collect())

In [ ]:
# 2: Выведите список исполнителей и количество их треков в порядке уменьшения.
# Ограничьте выборку исполнителями, у которых не менее 3 треков в наборе данных.

# +--------------+----------------+
# |   artist_name|number_of_tracks|
# +--------------+----------------+
# |      Doja Cat|               8|
# |        Coopex|               6|
# |     Dame Dame|               5|
# |     Lil Nas X|               4|
# |Alex Alexander|               4|
# |       YES YES|               4|
# |         Farux|               4|
# | Ariana Grande|               3|
# |       Astelle|               3|
# |  Alex D'Rosso|               3|
# |      Dua Lipa|               3|
# |    The Weeknd|               3|
# |     Nito-Onna|               3|
# |     Pop Smoke|               3|
# | The Kid LAROI|               3|
# +--------------+----------------+

In [ ]:
# 3: Выведите список названий треков, средняя громкость которых лежит в диапазоне от -10 до -9 Дб.

# ['Glimpse of Us', 'Left and Right (Charlie Puth) [feat. Jung Kook of BTS] - Sped Up Version',
# 'Frozen', "Can't Get You Out of My Head", 'I Hate U', 'I AM WOMAN', 'Buss It', 'Whatever You Like',
# 'MONEY', 'Freaks', 'telepatía', 'Falling Down', 'Party Girl', 'Skechers', 'Good Looking']

In [ ]:
#4: Выведите список треков, в названиях которых есть слово "love" (без учета регистра).

# ['Another Love', 'love nwantiti (ah ah ah)', 'Tainted Love', 'Looking for Love', 'Free Falling Love',
# 'Electric Love', 'Feel Your Love', 'What You Know Bout Love', 'Savage Love (Laxed - Siren Beat)',
# 'Love Me Again', 'Lovely']

In [ ]:
#5: Получите таблицу из песен Арианы Гранде и Astelle. Отсортируйте их по примеру ниже.

# +--------------------+-------------+
# |          track_name|  artist_name|
# +--------------------+-------------+
# |               34+35|Ariana Grande|
# |motive (with Doja...|Ariana Grande|
# |           positions|Ariana Grande|
# |             Dreamer|      Astelle|
# |           K Boy Bye|      Astelle|
# |                Paid|      Astelle|
# +--------------------+-------------+

In [ ]:
#6: Какую задачу решает приведенный ниже код?
# В чем суть метода K средних?
# Какие еще существуют методы машинного обучения для решения данной задачи?
# Какие выводы можно сделать о полученных группах?

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.sql import Row
from pyspark.ml.feature import MinMaxScaler
from IPython.display import display
import numpy as np

k = 20

np.set_printoptions(precision=3, suppress=True)

columns_to_scale = ["danceability", "energy", "speechiness", "acousticness",
                    "instrumentalness","liveness", "valence", "duration_ms",
                    "tempo", "track_pop"]
assemblers = [VectorAssembler(inputCols = [col], outputCol = col + "_vec") for col in columns_to_scale]
scalers = [MinMaxScaler(inputCol = col + "_vec", outputCol = col + "_scaled") for col in columns_to_scale]

vectorAssembler = VectorAssembler()\
  .setInputCols(["danceability_scaled", "energy_scaled", "speechiness_scaled", "acousticness_scaled",
                 "instrumentalness_scaled", "liveness_scaled", "valence_scaled", "duration_ms_scaled",
                 "tempo_scaled", "track_pop_scaled"])\
  .setOutputCol("features")

transformationPipeline = Pipeline()\
  .setStages(assemblers + scalers + [vectorAssembler])

fittedPipeline = transformationPipeline.fit(tiktokData2022)
transformedDF = fittedPipeline.transform(tiktokData2022)

kmeans = KMeans()\
  .setK(k)\
  .setSeed(300)

kmModel = kmeans.fit(transformedDF)

df_pred = kmModel.transform(transformedDF)

centers = kmModel.clusterCenters()
for i in range(0, k):
    print("group " + str(i))
    print(centers[i])
    df = df_pred.selectExpr("track_name", "danceability", "energy", "speechiness",
                        "acousticness","instrumentalness","liveness", "valence",
                        "duration_ms", "tempo", "track_pop")\
        .where("prediction = " + str(i))\
        .toPandas()
    display(df)